# 📊 Startup Data Analysis with MongoDB Aggregation Pipelines

This notebook demonstrates:
- Connecting to MongoDB Atlas
- Writing complex aggregation queries
- Extracting insights about funding trends, industries, and regions
- Performing exploratory data analysis

In [ ]:
import pandas as pd
import numpy as np
from pymongo import MongoClient
import json
from pprint import pprint
import sys

# Add parent directory to path
sys.path.append('..')
import config
from mongodb_setup import MongoDBHandler

## 1. Connect to MongoDB Atlas

In [ ]:
# Initialize MongoDB connection
handler = MongoDBHandler()
handler.connect()

# Get collection
collection = handler.db[config.COLLECTION_NAME]

# Verify connection
total_docs = collection.count_documents({})
print(f"\n📊 Total startups in database: {total_docs}")

# Display sample document
print("\n📄 Sample Document:")
sample = collection.find_one()
pprint(sample)

## 2. Aggregation Query #1: Top 10 Industries by Total Funding

Find which industries have received the most funding overall.

In [ ]:
pipeline_top_industries = [
    # Unwind the industry array to create separate documents for each industry
    {"$unwind": "$industry"},
    
    # Group by industry and sum total funding
    {
        "$group": {
            "_id": "$industry",
            "total_funding": {"$sum": "$total_funding_usd"},
            "startup_count": {"$sum": 1},
            "avg_funding": {"$avg": "$total_funding_usd"}
        }
    },
    
    # Sort by total funding descending
    {"$sort": {"total_funding": -1}},
    
    # Limit to top 10
    {"$limit": 10},
    
    # Project with formatted output
    {
        "$project": {
            "_id": 0,
            "industry": "$_id",
            "total_funding_usd": "$total_funding",
            "startup_count": 1,
            "avg_funding_usd": {"$round": ["$avg_funding", 2]}
        }
    }
]

print("🏆 Top 10 Industries by Total Funding:\n")
results = list(collection.aggregate(pipeline_top_industries))
df_industries = pd.DataFrame(results)

if not df_industries.empty:
    df_industries['total_funding_usd'] = df_industries['total_funding_usd'].apply(lambda x: f"${x:,.0f}")
    df_industries['avg_funding_usd'] = df_industries['avg_funding_usd'].apply(lambda x: f"${x:,.0f}")
    display(df_industries)
else:
    print("No data found")

## 3. Aggregation Query #2: Funding Distribution by Country

Analyze which countries have the most startup activity and funding.

In [ ]:
pipeline_by_country = [
    # Group by country
    {
        "$group": {
            "_id": "$country",
            "total_funding": {"$sum": "$total_funding_usd"},
            "startup_count": {"$sum": 1},
            "avg_funding": {"$avg": "$total_funding_usd"},
            "total_employees": {"$sum": "$employee_count"},
            "avg_employees": {"$avg": "$employee_count"}
        }
    },
    
    # Sort by total funding
    {"$sort": {"total_funding": -1}},
    
    # Format output
    {
        "$project": {
            "_id": 0,
            "country": "$_id",
            "total_funding_usd": "$total_funding",
            "startup_count": 1,
            "avg_funding_usd": {"$round": ["$avg_funding", 2]},
            "total_employees": 1,
            "avg_employees": {"$round": ["$avg_employees", 0]}
        }
    }
]

print("🌍 Funding Distribution by Country:\n")
results = list(collection.aggregate(pipeline_by_country))
df_countries = pd.DataFrame(results)

if not df_countries.empty:
    display(df_countries)
else:
    print("No data found")

## 4. Aggregation Query #3: Startups by Founding Year

Analyze startup formation trends over time.

In [ ]:
pipeline_by_year = [
    # Group by founding year
    {
        "$group": {
            "_id": "$founded_year",
            "startup_count": {"$sum": 1},
            "total_funding": {"$sum": "$total_funding_usd"},
            "avg_funding": {"$avg": "$total_funding_usd"}
        }
    },
    
    # Sort by year
    {"$sort": {"_id": 1}},
    
    # Format output
    {
        "$project": {
            "_id": 0,
            "year": "$_id",
            "startup_count": 1,
            "total_funding_usd": "$total_funding",
            "avg_funding_usd": {"$round": ["$avg_funding", 2]}
        }
    }
]

print("📅 Startup Formation Trends by Year:\n")
results = list(collection.aggregate(pipeline_by_year))
df_years = pd.DataFrame(results)

if not df_years.empty:
    display(df_years)
else:
    print("No data found")

## 5. Aggregation Query #4: Top Funded Startups

Find the startups with the highest total funding.

In [ ]:
pipeline_top_funded = [
    # Sort by total funding
    {"$sort": {"total_funding_usd": -1}},
    
    # Limit to top 10
    {"$limit": 10},
    
    # Project relevant fields
    {
        "$project": {
            "_id": 0,
            "name": 1,
            "country": 1,
            "industry": 1,
            "founded_year": 1,
            "total_funding_usd": 1,
            "employee_count": 1,
            "status": 1
        }
    }
]

print("💰 Top 10 Most Funded Startups:\n")
results = list(collection.aggregate(pipeline_top_funded))
df_top_funded = pd.DataFrame(results)

if not df_top_funded.empty:
    display(df_top_funded)
else:
    print("No data found")

## 6. Aggregation Query #5: Average Funding by Industry and Country

Cross-analyze industries and countries to find regional specializations.

In [ ]:
pipeline_industry_country = [
    # Unwind industries
    {"$unwind": "$industry"},
    
    # Group by industry and country
    {
        "$group": {
            "_id": {
                "industry": "$industry",
                "country": "$country"
            },
            "startup_count": {"$sum": 1},
            "avg_funding": {"$avg": "$total_funding_usd"},
            "total_funding": {"$sum": "$total_funding_usd"}
        }
    },
    
    # Filter for meaningful data (at least 2 startups)
    {"$match": {"startup_count": {"$gte": 1}}},
    
    # Sort by total funding
    {"$sort": {"total_funding": -1}},
    
    # Limit results
    {"$limit": 15},
    
    # Format output
    {
        "$project": {
            "_id": 0,
            "industry": "$_id.industry",
            "country": "$_id.country",
            "startup_count": 1,
            "avg_funding_usd": {"$round": ["$avg_funding", 2]},
            "total_funding_usd": "$total_funding"
        }
    }
]

print("🌐 Industry-Country Analysis:\n")
results = list(collection.aggregate(pipeline_industry_country))
df_industry_country = pd.DataFrame(results)

if not df_industry_country.empty:
    display(df_industry_country)
else:
    print("No data found")

## 7. Aggregation Query #6: Startup Status Distribution

Analyze the current status of startups (Operating, Acquired, Closed, etc.)

In [ ]:
pipeline_status = [
    # Group by status
    {
        "$group": {
            "_id": "$status",
            "count": {"$sum": 1},
            "total_funding": {"$sum": "$total_funding_usd"},
            "avg_funding": {"$avg": "$total_funding_usd"}
        }
    },
    
    # Sort by count
    {"$sort": {"count": -1}},
    
    # Format output
    {
        "$project": {
            "_id": 0,
            "status": "$_id",
            "count": 1,
            "total_funding_usd": "$total_funding",
            "avg_funding_usd": {"$round": ["$avg_funding", 2]}
        }
    }
]

print("📊 Startup Status Distribution:\n")
results = list(collection.aggregate(pipeline_status))
df_status = pd.DataFrame(results)

if not df_status.empty:
    display(df_status)
else:
    print("No data found")

## 8. Complex Query: Top Cities for Startups

Find startup hubs with the most activity.

In [ ]:
pipeline_cities = [
    # Group by city and country
    {
        "$group": {
            "_id": {
                "city": "$city",
                "country": "$country"
            },
            "startup_count": {"$sum": 1},
            "total_funding": {"$sum": "$total_funding_usd"},
            "total_employees": {"$sum": "$employee_count"}
        }
    },
    
    # Sort by startup count
    {"$sort": {"startup_count": -1}},
    
    # Limit to top 10
    {"$limit": 10},
    
    # Format output
    {
        "$project": {
            "_id": 0,
            "city": "$_id.city",
            "country": "$_id.country",
            "startup_count": 1,
            "total_funding_usd": "$total_funding",
            "total_employees": 1
        }
    }
]

print("🏙️ Top Startup Cities:\n")
results = list(collection.aggregate(pipeline_cities))
df_cities = pd.DataFrame(results)

if not df_cities.empty:
    display(df_cities)
else:
    print("No data found")

## 9. Summary Statistics

In [ ]:
# Overall statistics
pipeline_summary = [
    {
        "$group": {
            "_id": None,
            "total_startups": {"$sum": 1},
            "total_funding": {"$sum": "$total_funding_usd"},
            "avg_funding": {"$avg": "$total_funding_usd"},
            "max_funding": {"$max": "$total_funding_usd"},
            "min_funding": {"$min": "$total_funding_usd"},
            "total_employees": {"$sum": "$employee_count"},
            "avg_employees": {"$avg": "$employee_count"}
        }
    }
]

summary = list(collection.aggregate(pipeline_summary))[0]

print("\n" + "="*60)
print("📊 OVERALL STARTUP ECOSYSTEM SUMMARY")
print("="*60)
print(f"\n🏢 Total Startups: {summary['total_startups']:,}")
print(f"💰 Total Funding: ${summary['total_funding']:,.0f}")
print(f"📈 Average Funding: ${summary['avg_funding']:,.0f}")
print(f"🎯 Max Funding: ${summary['max_funding']:,.0f}")
print(f"📉 Min Funding: ${summary['min_funding']:,.0f}")
print(f"👥 Total Employees: {summary['total_employees']:,}")
print(f"👤 Avg Employees per Startup: {summary['avg_employees']:.0f}")
print("\n" + "="*60)

## 10. Export Results for Visualization

In [ ]:
import os

# Save all analysis results
output_dir = config.PROCESSED_DATA_DIR

if not df_industries.empty:
    df_industries.to_csv(os.path.join(output_dir, 'top_industries.csv'), index=False)
    
if not df_countries.empty:
    df_countries.to_csv(os.path.join(output_dir, 'funding_by_country.csv'), index=False)
    
if not df_years.empty:
    df_years.to_csv(os.path.join(output_dir, 'startups_by_year.csv'), index=False)
    
if not df_top_funded.empty:
    df_top_funded.to_csv(os.path.join(output_dir, 'top_funded_startups.csv'), index=False)

print("✅ Analysis results exported to data/processed/")
print("\n📊 Ready for visualization in visualization.ipynb")

## Key Insights

Based on the aggregation queries above, we can derive:

1. **Industry Trends**: Which sectors are attracting the most capital
2. **Geographic Patterns**: Regional innovation hotspots and funding distribution
3. **Temporal Analysis**: How startup formation has evolved over time
4. **Success Metrics**: Characteristics of highly-funded startups
5. **Market Dynamics**: Industry-country specializations

## Next Steps

Proceed to `visualization.ipynb` to create compelling visualizations of these insights.

In [ ]:
# Close MongoDB connection
handler.close()